# SDB Benchmark

## Choices

We took several parameters into account in order to provide unbiased and reproductible results. We present them in the next section.

### Time measurement

In order to measure accurately the time taken for a database CRUD operation to complete, we need to measure the duration between the moment the command is called and the moment it returns (both postgres and cassandra python API functions returns synchronuously). In addition, we measure time by several (100) batches of 100 measurements. This allows to plot the distribution of several latencies AND have precise enough measurements. 
Besides, we use the time.perf_counter function from the time module which has the best accuracy for measuring time in Python.Finally, we also clean the built-in garbage collector of Python before each measurement to ensure consistent behaviour.

### Reproducibility

Most of the queries we generate are random to limit bias in the analysis : one database system could perform surprisingly well for a specific data. Using random queries reduce such a bias. Therefore, we chose to write our queries once to a file and to perform them by reading directly the file instead of generating news random datas at each runtime execution. This let us make precise commentaries on the actual data we have observed which is the very same data you will see being displayed.

### Displaying results

We choose to display results with histograms to overview the whole frequency distribution rather than only the mean or the standard deviation. When needed, both means are represented alongside to better compare the results.

## Initialise cassandra database

If the following cells raise a `ConnexionRefuseError`, it may be because of the Cassandra database didn't finished to initialize.

In [7]:
import csv, gc, time
import numpy as np
from matplotlib import pyplot as plt 
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

In [8]:
time.sleep(120)

cluster = Cluster(['cassandra'])
session = cluster.connect()

# Create a keyspace and table
session.execute("""CREATE KEYSPACE IF NOT EXISTS benchmarking WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : '1' }""")
session.execute("""CREATE TABLE IF NOT EXISTS benchmarking.bitcoin_addresses (BITCOIN_ADDRESS text, ACCOUNT text, 
IP_ADDRESS text, COUNTY text, COUNTRY_CODE text, DATABASE_COLUMN_TYPE text, PRIMARY KEY 
(BITCOIN_ADDRESS))""")


## Import Data into cassandra

In [ ]:
# import data
with open("/home/data.csv", 'r') as file:
    reader = csv.DictReader(file)
    insert_query = session.prepare("""
INSERT INTO benchmarking.bitcoin_addresses (BITCOIN_ADDRESS, ACCOUNT, IP_ADDRESS, COUNTY, COUNTRY_CODE, DATABASE_COLUMN_TYPE)
VALUES (?, ?, ?, ?, ?, ?)
""")
    cassandra_times_import = np.empty(100)

    for sample in range(100):
        batch = BatchStatement()

        for i in range(1000):
            row = next(reader)
            session.execute(insert_query, (
                row['BITCOIN_ADDRESS'],
                row['ACCOUNT'],
                row['IP_ADDRESS'],
                row['COUNTY'],
                row['COUNTRY_CODE'],
                row['DATABASE_COLUMN_TYPE']
            ))

        gc.collect()
        start_time = time.perf_counter()

        session.execute(batch)

        end_time = time.perf_counter()
        cassandra_times_import[sample] = (end_time - start_time)
        print(sample, "%")

## Initialise PostrgreSQL database

In [5]:
# Connect to the database and set up the table
import psycopg2, random, string
from psycopg2 import sql

connection = psycopg2.connect(
    dbname="postgresDB", 
    user="user", 
    password="tprli", 
    host="postgres",  # Or use your database host if it's not local
    port="5432"  # Default PostgreSQL port
)
connection.autocommit = True
cursor = connection.cursor()


cursor.execute("DROP TABLE IF EXISTS bitcoin_addresses;")

# Step 2: Create the table in PostgreSQL if it doesn't exist
create_table_query = """
CREATE TABLE bitcoin_addresses (
    bitcoin_address VARCHAR(34) NOT NULL, -- Bitcoin addresses are usually 26-35 characters
    account VARCHAR(20),                 -- Account numbers, max length derived from the example
    ip_address VARCHAR(15),              -- IPv4 addresses, formatted as strings
    county VARCHAR(50),                  -- County names, variable length
    country_code CHAR(2),                -- Two-letter country codes
    database_column_type VARCHAR(50),   -- Describes column type as text (e.g., mediumint, float),
    PRIMARY KEY (bitcoin_address)
);
"""
cursor.execute(create_table_query)

## import data into PostgreSQL

In [ ]:
postgres_times_import = np.empty(100)
with open('/home/data.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row


    insert_query = """
        INSERT INTO bitcoin_addresses (bitcoin_address, account, ip_address, county, country_code, database_column_type) VALUES (%s, %s, %s, %s, %s, %s)
        """


    for i in range(100):
        gc.collect()
        start_time = time.perf_counter()
        for j in range(1000):
            cursor.execute(insert_query, next(reader))
        end_time = time.perf_counter()
        postgres_times_import[i] = (end_time - start_time)
        print(i, "%")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

min_bin = min(cassandra_times_import.min(), postgres_times_import.min())
max_bin = max(cassandra_times_import.max(), postgres_times_import.max())
bins = np.linspace(min_bin, max_bin, 200)  # 20 bins entre les valeurs min et max

# Création de l'histogramme avec des bornes communes
plt.figure(figsize=(10, 6))
plt.hist(cassandra_times_import, bins=bins, alpha=0.7, label='Cassandra', color='blue')
plt.hist(postgres_times_import, bins=bins, alpha=0.7, label='Postgresql', color='orange')

# Ajout des légendes et des titres
plt.title('Histogram of import times with 2 databases systems', fontsize=16)
plt.xlabel('SGBD Systems', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)

# Affichage
plt.show()

In [ ]:
moyenne1 = np.mean(cassandra_times_import)
moyenne2 = np.mean(postgres_times_import)

labels = ['Cassandra', 'PostgreSQL']
moyennes = [moyenne1, moyenne2]

# Création du graphique en barres
plt.figure(figsize=(8, 6))
plt.bar(labels, moyennes, color=['blue', 'orange'], alpha=0.7)

# Ajout des annotations
for i, moyenne in enumerate(moyennes):
    plt.text(i, moyenne + 1, f'{moyenne:.8f}', ha='center', fontsize=12)

# Ajout des titres et légendes
plt.title('Mean of import durations', fontsize=16)
plt.ylabel('Mean of import durations', fontsize=14)
plt.grid(axis='y', alpha=0.3)

# Affichage
plt.show()

## Requests Generation

Generation of randomized queries

In [ ]:
import random
# Generate random requests
simulated_values = {
    'BITCOIN_ADDRESS': lambda: f"'{''.join(random.choices('ABCDEFGHJKLMNPQRSTUVWXYZ123456789', k=32))}'",
    'ACCOUNT': lambda: f"'{random.randint(10000000, 99999999)}'",
    'IP_ADDRESS': lambda: f"'{random.randint(1, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(1, 255)}'",
    'COUNTY': lambda: f"'{random.choice(['Buckinghamshire', 'Avon', 'Cambridgeshire', 'Bedfordshire', 'Borders'])}'",
    'COUNTRY_CODE': lambda: f"'{random.choice('ABCDEFGHJKLMNPQRSTUVWXYZ')}{random.choice('ABCDEFGHJKLMNPQRSTUVWXYZ')}'",
    'DATABASE_COLUMN_TYPE': lambda: f"'{random.choice(['float', 'point', 'int', 'serial', 'varchar', 'blob', 'timestamp'])}'"
}
def generate_random_select_primary_key(table_name, primary_key, set_of_primary_key_values):
    primary_key_value = random.choice(set_of_primary_key_values)

    select_query = f"SELECT * FROM {table_name} WHERE {primary_key} = '{primary_key_value}'"
    return select_query

def generate_random_select_conditions(tables_name, column_names):
    column = random.choice(column_names)

    value = simulated_values[column]()  # Générer une valeur simulée pour la colonne
    where_clause = f"WHERE {column} = {value}"

    select_queries = {}
    for table in tables_name:
        select_queries[table] = f"SELECT {column} FROM {table} {where_clause}"
    return select_queries

def generate_random_update_query(table_name, column_names, primary_key, set_of_primary_key_values):
    num_updates = random.randint(1,3)
    columns_to_update = random.sample(column_names, num_updates)
    update_values = [f"{col} = {simulated_values[col]()}" for col in columns_to_update]

    primary_key_value = random.choice(set_of_primary_key_values)

    update_query = f"UPDATE {table_name} SET {', '.join(update_values)} WHERE {primary_key} = '{primary_key_value}'"
    return update_query

def generate_random_delete_query(table_name, primary_key, set_of_primary_key_values):
    primary_key_value = random.choice(set_of_primary_key_values)

    delete_query = f"DELETE FROM {table_name} WHERE {primary_key} = '{primary_key_value}'"
    return delete_query

table = "bitcoin_addresses"
table_indexed = "bitcoin_addresses_indexed"
table_not_indexed = "bitcoin_addresses_not_indexed"
conditions_column = ['COUNTY', 'COUNTRY_CODE','DATABASE_COLUMN_TYPE']
update_column = ['ACCOUNT', 'IP_ADDRESS', 'COUNTY', 'COUNTRY_CODE', 'DATABASE_COLUMN_TYPE']
primary_key = 'BITCOIN_ADDRESS'
set_of_key = []
rows = session.execute("SELECT BITCOIN_ADDRESS FROM benchmarking.bitcoin_addresses")
for a in rows:
    set_of_key.append(a[0])

# Generate Select queries to match primary key
with open("select_primary_key.sql", "w") as f:
    for i in range(10000):
        f.write(generate_random_select_primary_key(table, primary_key, set_of_key))
        f.write("\n")
    f.close()
    
# Generate Select with conditions queries
with  open("select_conditions_not_indexed.sql", "w") as f, open("select_conditions_indexed.sql", "w") as f_index_table, open("select_conditions_postgre.sql", "w") as f_postgre:
    for i in range(100):
        queries = generate_random_select_conditions([table_not_indexed, table_indexed, table], conditions_column)
        f.write(queries[table_not_indexed])
        f.write("\n")
        f_index_table.write(queries[table_indexed])
        f_index_table.write("\n")
        f_postgre.write(queries[table])
        f_postgre.write("\n")
    f.close()

# Generate Update queries
with open("update.sql", "w") as f:
    for i in range(10000):
        f.write(generate_random_update_query(table, update_column, primary_key, set_of_key))
        f.write("\n")
    f.close()

# Generate Delete queries
with open("delete.sql", "w") as f:
    for i in range(10000):
        f.write(generate_random_delete_query(table, primary_key, set_of_key))
        f.write("\n")
    f.close()

## Queries Benchmarking

### Select


**Cassandra**

In [ ]:
# Benchmark SELECT
session.set_keyspace("benchmarking")

select_path = "select_primary_key_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = int(len(lines)/100)
    cassandra_times_select = np.empty(n_iter)
    
    for i in range(n_iter):
        gc.collect()
        start_time = time.perf_counter()

        for j in range(100):
            session.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        cassandra_times_select[i] = (end_time - start_time)

print(cassandra_times_select)

**PostgreSQL**

In [ ]:
select_path = "select_primary_key_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = int(len(lines)/100)
    postgres_times_select = np.empty(n_iter)
    
    for i in range(n_iter):
        gc.collect()
        start_time = time.perf_counter()

        for j in range(100):
            cursor.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        postgres_times_select[i] = (end_time - start_time)
print(postgres_times_select)

**Analysis**

In [ ]:
min_bin = min(cassandra_times_select.min(), postgres_times_select.min())
max_bin = max(cassandra_times_select.max(), postgres_times_select.max())
bins = np.linspace(min_bin, max_bin, 200)  # 20 bins entre les valeurs min et max

# Création de l'histogramme avec des bornes communes
plt.figure(figsize=(10, 6))
plt.hist(cassandra_times_select, bins=bins, alpha=0.7, label='Cassandra', color='blue')
plt.hist(postgres_times_select, bins=bins, alpha=0.7, label='PostgreSQL', color='orange')

# Ajout des légendes et des titres
plt.title('Comparison of select queries', fontsize=16)
plt.xlabel('SGBD system', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)

# Affichage
plt.show()

### Update


**Cassandra**

In [ ]:
# Benchmark Update
session.set_keyspace("benchmarking")

select_path = "update_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = int(len(lines)/100)
    cassandra_times_update = np.empty(n_iter)
    
    for i in range(n_iter):
        gc.collect()
        start_time = time.perf_counter()

        for j in range(100):
            session.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        cassandra_times_update[i] = (end_time - start_time)
print(cassandra_times_update)

**PostgreSQL**

In [ ]:
select_path = "update_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = int(len(lines)/100)
    postgres_times_update = np.empty(n_iter)
    
    for i in range(n_iter):
        gc.collect()
        start_time = time.perf_counter()

        for j in range(100):
            cursor.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        postgres_times_update[i] = (end_time - start_time)
print(postgres_times_update)

**Analysis**

In [ ]:
min_bin = min(cassandra_times_update.min(), postgres_times_update.min())
max_bin = max(cassandra_times_update.max(), postgres_times_update.max())
bins = np.linspace(min_bin, max_bin, 200)  # 20 bins entre les valeurs min et max

# Création de l'histogramme avec des bornes communes
plt.figure(figsize=(10, 6))
plt.hist(cassandra_times_update, bins=bins, alpha=0.7, label='Cassandra', color='blue')
plt.hist(postgres_times_update, bins=bins, alpha=0.7, label='PostgreSQL', color='orange')

# Ajout des légendes et des titres
plt.title('Comparison of update queries', fontsize=16)
plt.xlabel('SGBD system', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)

# Affichage
plt.show()

### Delete


**Cassandra**

In [ ]:
# Benchmark DELETE
session.set_keyspace("benchmarking")

select_path = "delete_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = int(len(lines)/100)
    cassandra_times_delete = np.empty(n_iter)
    
    for i in range(n_iter):
        gc.collect()
        start_time = time.perf_counter()

        for j in range(100):
            session.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        cassandra_times_delete[i] = (end_time - start_time)
print(cassandra_times_delete)

**PostgreSQL**

In [ ]:
# Benchmark DELETE

select_path = "delete_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = int(len(lines)/100)
    postgres_times_delete = np.empty(n_iter)
    
    for i in range(n_iter):
        gc.collect()
        start_time = time.perf_counter()

        for j in range(100):
            cursor.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        postgres_times_delete[i] = (end_time - start_time)
print(postgres_times_delete)

**Analysis**

In [ ]:
min_bin = min(cassandra_times_delete.min(), postgres_times_delete.min())
max_bin = max(cassandra_times_delete.max(), postgres_times_delete.max())
bins = np.linspace(min_bin, max_bin, 200)  # 20 bins entre les valeurs min et max

# Création de l'histogramme avec des bornes communes
plt.figure(figsize=(10, 6))
plt.hist(cassandra_times_delete, bins=bins, alpha=0.7, label='Cassandra', color='blue')
plt.hist(postgres_times_delete, bins=bins, alpha=0.7, label='PostgreSQL', color='orange')

# Ajout des légendes et des titres
plt.title('Comparison of delete queries', fontsize=16)
plt.xlabel('SGBD system', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)

# Affichage
plt.show()

### Conclusion about postgreSQL vs cassandra

We observe that Cassandra is faster than PostgreSQL for all CRUD operations, especially for import where Cassandra is more than 3000 times faster. It can be explained by the fact it is presented as a write-oriented NoSQL system, especially suited for such operations.

## Index utilisation

### Initialize

Set up new table to benchmark effect of indexing.

In [ ]:
# Create two tables to compare 
session.execute("""CREATE TABLE IF NOT EXISTS benchmarking.bitcoin_addresses_not_indexed (BITCOIN_ADDRESS text, ACCOUNT text, 
IP_ADDRESS text, COUNTY text, COUNTRY_CODE text, DATABASE_COLUMN_TYPE text, PRIMARY KEY 
(BITCOIN_ADDRESS))""")
session.execute("""CREATE TABLE IF NOT EXISTS benchmarking.bitcoin_addresses_indexed (BITCOIN_ADDRESS text, ACCOUNT text, 
IP_ADDRESS text, COUNTY text, COUNTRY_CODE text, DATABASE_COLUMN_TYPE text, PRIMARY KEY 
(BITCOIN_ADDRESS))""")

# Import data
with open("/home/data.csv", 'r') as file:
    reader = csv.DictReader(file)
    notindexed_insert_query = session.prepare("""
INSERT INTO benchmarking.bitcoin_addresses_indexed (BITCOIN_ADDRESS, ACCOUNT, IP_ADDRESS, COUNTY, COUNTRY_CODE, DATABASE_COLUMN_TYPE)
VALUES (?, ?, ?, ?, ?, ?)
""")

    batch = BatchStatement()

    for row in reader:
        session.execute(notindexed_insert_query, (
            row['BITCOIN_ADDRESS'],
            row['ACCOUNT'],
            row['IP_ADDRESS'],
            row['COUNTY'],
            row['COUNTRY_CODE'],
            row['DATABASE_COLUMN_TYPE']
        ))

    session.execute(batch)

**Creation of an index**

`COUNTY` `COUNTRY_CODE` `DATABASE_COLUMN_TYPE` columns are selected to be indexed because they have a moderate cardinality. The indexing must have a significant effect on queries performances.

In [15]:

# Create index
create_index_query_county = """CREATE INDEX IF NOT EXISTS countyindex on benchmarking.bitcoin_addresses_indexed (COUNTY)"""
session.execute(create_index_query_county)
create_index_query_countrycode = """CREATE INDEX IF NOT EXISTS countrycodeindex on benchmarking.bitcoin_addresses_indexed (COUNTRY_CODE)"""
session.execute(create_index_query_countrycode)
create_index_query_datacolumntype = """CREATE INDEX IF NOT EXISTS datacolumntypeindex on benchmarking.bitcoin_addresses_indexed (DATABASE_COLUMN_TYPE)"""
session.execute(create_index_query_datacolumntype)

**Import with indexes**

Indexing impacts import performance because the database must update the index structure.

In [ ]:
# Import data
with open("/home/data.csv", 'r') as file:
    reader = csv.DictReader(file)
    insert_query = session.prepare("""
INSERT INTO benchmarking.bitcoin_addresses_indexed (BITCOIN_ADDRESS, ACCOUNT, IP_ADDRESS, COUNTY, COUNTRY_CODE, DATABASE_COLUMN_TYPE)
VALUES (?, ?, ?, ?, ?, ?)
""")
    cassandra_indexed_times_import = np.empty(100)

    for sample in range(100):
        batch = BatchStatement()

        for i in range(1000):
            row = next(reader)
            session_replicate.execute(insert_query, (
                row['BITCOIN_ADDRESS'],
                row['ACCOUNT'],
                row['IP_ADDRESS'],
                row['COUNTY'],
                row['COUNTRY_CODE'],
                row['DATABASE_COLUMN_TYPE']
            ))

        gc.collect()
        start_time = time.perf_counter()

        session_replicate.execute(batch)

        end_time = time.perf_counter()
        cassandra_indexed_times_import[sample] = (end_time - start_time)
        print(sample, "%")

### Queries with conditions
To evaluate the performance of indexing, we perform SELECT queries with where clause on indexed columns.

**Indexed table**

In [34]:
# Benchmark indexed table
session.set_keyspace("benchmarking")

select_path = "select_conditions_indexed_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = len(lines)
    cassandra_times_select_indexed = np.empty(n_iter)
    
    for line in lines:
        gc.collect()
        start_time = time.perf_counter()

        session.execute(line)

        end_time = time.perf_counter()
        cassandra_times_select_indexed[i] = (end_time - start_time)

**Not indexed table**

In [ ]:
# Benchmark not indexed table
session.set_keyspace("benchmarking")

select_path = "select_conditions_not_indexed_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = len(lines)
    cassandra_times_select_not_indexed = np.empty(n_iter)
    for i in range(n_iter):
        lines[i] = f"{lines[i][:-1]} ALLOW FILTERING"
    
    for line in lines:
        gc.collect()
        start_time = time.perf_counter()

        session.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        cassandra_times_select_not_indexed[i] = (end_time - start_time)

**PostgreSQL**

In [ ]:
# Benchmark not indexed table postgres

select_path = "select_conditions_not_indexed_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = len(lines)
    postgres_times_select_not_indexed = np.empty(n_iter)
    
    for line in lines:
        gc.collect()
        start_time = time.perf_counter()

        cursor.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        postgres_times_select_not_indexed[i] = (end_time - start_time)

## Replication performances benchmarking

**Setup replication**

In [2]:
cluster_replicate = Cluster(['cassandra-replicat-0'])
session_replicate = cluster_replicate.connect()

# Create keyspace with replication
session_replicate.execute("""CREATE KEYSPACE IF NOT EXISTS benchmarking WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : '3' }""")

# Create table
session_replicate.execute("""CREATE TABLE IF NOT EXISTS benchmarking.bitcoin_addresses (BITCOIN_ADDRESS text, ACCOUNT text, 
IP_ADDRESS text, COUNTY text, COUNTRY_CODE text, DATABASE_COLUMN_TYPE text, PRIMARY KEY 
(BITCOIN_ADDRESS))""")

**Import Data**

In [ ]:
# Import data
with open("/home/data.csv", 'r') as file:
    reader = csv.DictReader(file)
    insert_query = session_replicate.prepare("""
INSERT INTO benchmarking.bitcoin_addresses (BITCOIN_ADDRESS, ACCOUNT, IP_ADDRESS, COUNTY, COUNTRY_CODE, DATABASE_COLUMN_TYPE)
VALUES (?, ?, ?, ?, ?, ?)
""")
    cassandra_replication_times_import = np.empty(100)

    for sample in range(100):
        batch = BatchStatement()

        for i in range(1000):
            row = next(reader)
            session_replicate.execute(insert_query, (
                row['BITCOIN_ADDRESS'],
                row['ACCOUNT'],
                row['IP_ADDRESS'],
                row['COUNTY'],
                row['COUNTRY_CODE'],
                row['DATABASE_COLUMN_TYPE']
            ))

        gc.collect()
        start_time = time.perf_counter()

        session_replicate.execute(batch)

        end_time = time.perf_counter()
        cassandra_replication_times_import[sample] = (end_time - start_time)
        print(sample, "%")

**SELECT**

In [4]:
# Benchmark SELECT
session_replicate.set_keyspace("benchmarking")

select_path = "select_primary_key_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = int(len(lines)/100)
    cassandra_replicate_times_select = np.empty(n_iter)
    
    for i in range(n_iter):
        gc.collect()
        start_time = time.perf_counter()

        for j in range(100):
            session_replicate.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        cassandra_replicate_times_select[i] = (end_time - start_time)

**UPDATE**

In [5]:
# Benchmark DELETE
session_replicate.set_keyspace("benchmarking")

select_path = "update_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = int(len(lines)/100)
    cassandra_replicate_times_update = np.empty(n_iter)
    
    for i in range(n_iter):
        gc.collect()
        start_time = time.perf_counter()

        for j in range(100):
            session_replicate.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        cassandra_replicate_times_update[i] = (end_time - start_time)

**Delete**

In [6]:
# Benchmark DELETE
session_replicate.set_keyspace("benchmarking")

select_path = "delete_0.sql"
with open(select_path, "r") as file:
    lines = file.readlines()
    n_iter = int(len(lines)/100)
    cassandra_replicate_times_delete = np.empty(n_iter)
    
    for i in range(n_iter):
        gc.collect()
        start_time = time.perf_counter()

        for j in range(100):
            session_replicate.execute(lines[i*100 + j])

        end_time = time.perf_counter()
        cassandra_replicate_times_delete[i] = (end_time - start_time)